<a href="https://colab.research.google.com/github/Jorgeduardo13/SistemasInteligentes/blob/main/Challenge_2_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar librerias

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

pd.options.display.max_colwidth = 100

from google.colab import drive
drive.mount('/content/drive')

#Plotly
import plotly.offline as py #visualization
import plotly.graph_objs as go
from plotly.offline import *

from sklearn.model_selection import train_test_split
from sklearn import metrics


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Lectura de datos

Los datos leidos ya estan estandarizados usando un proceso de minmax() y ya tiene los datos faltantes imputados con missforrest(se uso un algoritmo de random forrest).

In [ ]:
df = pd.read_csv(r'/content/drive/MyDrive/Sistemas Inteligentes/train_clean.csv', index_col=0)

In [ ]:
df

,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class,Class_name
0,Bruno Mars,That's What I Like (feat. Gucci Mane),0.595960,0.854745,0.563991,0.0,0.847024,1.0,0.027882,0.017169,0.097608,0.073879,0.910096,0.553969,0.158812,0.75,5,HipHop
1,Boston,Hitch a Ride,0.535354,0.346890,0.813996,0.2,0.792166,1.0,0.019410,0.001104,0.004025,0.090173,0.569081,0.459689,0.170413,0.75,10,Country
2,The Raincoats,No Side to Fall In,0.343434,0.402841,0.613992,0.5,0.765439,1.0,0.032172,0.487952,0.000196,0.386702,0.794358,0.626804,0.074240,0.75,6,Alt_Music
3,Deno,Lingo (feat. J.I & Chunkz),0.656566,0.853669,0.596992,0.9,0.809161,0.0,0.035389,0.021285,0.053236,0.111426,0.569081,0.409271,0.117769,0.75,5,HipHop
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,0.525253,0.115558,0.974999,0.1,0.863607,1.0,0.207507,0.000170,0.016164,0.162028,0.075953,0.901766,0.155674,0.75,10,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17991,Green-House,Find Home,0.343434,0.114482,0.108982,0.6,0.553223,0.0,0.020161,0.996988,0.827309,0.087542,0.163997,0.754740,0.130958,0.50,6,Alt_Music
17992,Micatone,All Gone,0.262626,0.622337,0.222984,1.0,0.720895,0.0,0.011153,0.861446,0.000015,0.059306,0.327271,0.227225,0.174024,0.75,2,Alt
17993,Smash Hit Combo,Peine perdue,0.333333,0.536260,0.981000,0.3,0.853826,0.0,0.052225,0.000030,0.000136,0.661978,0.251834,0.398391,0.146374,0.75,8,Acoustic/Folk
17994,Beherit,Salomon's Gate,0.282828,0.167205,0.804996,0.5,0.658363,0.0,0.119571,0.001295,0.919679,0.247040,0.347938,0.539476,0.148724,0.75,8,Acoustic/Folk


In [ ]:
cols=['Popularity', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo',
       'duration_in min/ms', 'time_signature']

In [ ]:
train_X_NotText,test_X_NotText,train_y_NotText,test_y_NotText = train_test_split(df[cols],df['Class'],test_size = .3,random_state=0 )

In [ ]:
class_mapping={0:'Rock', 1:'Indie', 2:'Alt', 3:'Pop',4: 'Metal', 5:'HipHop',6: 'Alt_Music', 7:'Blues',
               8:'Acoustic/Folk', 9:'Instrumental', 10:'Country',11: 'Bollywood'}
df


,Artist Name,Track Name,Popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_in min/ms,time_signature,Class,Class_name
0,Bruno Mars,That's What I Like (feat. Gucci Mane),0.595960,0.854745,0.563991,0.0,0.847024,1.0,0.027882,0.017169,0.097608,0.073879,0.910096,0.553969,0.158812,0.75,5,HipHop
1,Boston,Hitch a Ride,0.535354,0.346890,0.813996,0.2,0.792166,1.0,0.019410,0.001104,0.004025,0.090173,0.569081,0.459689,0.170413,0.75,10,Country
2,The Raincoats,No Side to Fall In,0.343434,0.402841,0.613992,0.5,0.765439,1.0,0.032172,0.487952,0.000196,0.386702,0.794358,0.626804,0.074240,0.75,6,Alt_Music
3,Deno,Lingo (feat. J.I & Chunkz),0.656566,0.853669,0.596992,0.9,0.809161,0.0,0.035389,0.021285,0.053236,0.111426,0.569081,0.409271,0.117769,0.75,5,HipHop
4,Red Hot Chili Peppers,Nobody Weird Like Me - Remastered,0.525253,0.115558,0.974999,0.1,0.863607,1.0,0.207507,0.000170,0.016164,0.162028,0.075953,0.901766,0.155674,0.75,10,Country
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17991,Green-House,Find Home,0.343434,0.114482,0.108982,0.6,0.553223,0.0,0.020161,0.996988,0.827309,0.087542,0.163997,0.754740,0.130958,0.50,6,Alt_Music
17992,Micatone,All Gone,0.262626,0.622337,0.222984,1.0,0.720895,0.0,0.011153,0.861446,0.000015,0.059306,0.327271,0.227225,0.174024,0.75,2,Alt
17993,Smash Hit Combo,Peine perdue,0.333333,0.536260,0.981000,0.3,0.853826,0.0,0.052225,0.000030,0.000136,0.661978,0.251834,0.398391,0.146374,0.75,8,Acoustic/Folk
17994,Beherit,Salomon's Gate,0.282828,0.167205,0.804996,0.5,0.658363,0.0,0.119571,0.001295,0.919679,0.247040,0.347938,0.539476,0.148724,0.75,8,Acoustic/Folk


# Algoritmos Géneticos

Se usara un algoritmo génetico para definir el clasificador que maximize las métricas de desempeño del algoritmo. 

1.   Modelo Base : 0.33 - Naive Bayes
2.   Mejor Modelo : 0.57 - Random Forrest con tfidf en el titulo de las canciones.




In [ ]:
! pip install tpot

In [ ]:
from tpot import TPOTClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning:

The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.



## First Try

In [ ]:
tpot = TPOTClassifier(
    #config_dict='TPOT sparse',
    generations=10, population_size=5,
    scoring = 'f1_macro',
    max_time_mins = 30, 
    n_jobs = -1,
    early_stop=10,
    verbosity = 3,
    cv = 3,
    random_state=0
    )

In [ ]:
tpot.fit(train_X_NotText, train_y_NotText)

32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/5 [00:00<?, ?pipeline/s]


Generation 1 - Current Pareto front scores:

-1	0.5349034055215142	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBClassifier__min_child_weight=7, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.4, XGBClassifier__verbosity=0)

Generation 2 - Current Pareto front scores:

-1	0.5368539212461337	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBClassifier__min_child_weight=7, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.45, XGBClassifier__verbosity=0)

-2	0.53979290797925	RandomForestClassifier(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, PolynomialFeatures__include_bias=False, PolynomialFeatures__interaction_only=False), RandomForestClassifier__bootstrap=False, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.3, RandomForestClassifier__min_samples_leaf=1, RandomForestClassifier

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=3,
               disable_update_check=False, early_stop=10, generations=10,
               log_file=None, max_eval_time_mins=5, max_time_mins=30,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=5,
               random_state=0, scoring='f1_macro', subsample=1.0, template=None,
               use_dask=False, verbosity=3, warm_start=False)

In [ ]:
pd.DataFrame(tpot.evaluated_individuals_).T.reset_index().sort_values(by='internal_cv_score',ascending=False)

,index,generation,mutation_count,crossover_count,predecessor,operator_count,internal_cv_score
34,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=9, XGBCla...",7,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=9, XGBCl...",1,0.550757
35,"XGBClassifier(GaussianNB(input_matrix), XGBClassifier__learning_rate=0.1, XGBClassifier__max_dep...",7,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=8, XGBCl...",2,0.550556
38,"XGBClassifier(MaxAbsScaler(input_matrix), XGBClassifier__learning_rate=0.1, XGBClassifier__max_d...",7,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=9, XGBCl...",2,0.550404
23,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=9, XGBCla...",4,1,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.54894
28,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCla...",5,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCl...",1,0.548857
18,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=8, XGBCla...",3,1,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.548435
20,"XGBClassifier(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, PolynomialFeatures_...",4,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",2,0.547411
37,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCla...",7,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCl...",1,0.547052
15,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCla...",3,1,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.546558
39,"XGBClassifier(ZeroCount(input_matrix), XGBClassifier__learning_rate=0.5, XGBClassifier__max_dept...",8,3,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCl...",2,0.546451


In [ ]:
tpot.export('/content/drive/MyDrive/Sistemas Inteligentes/BestModel_gen10_pop5.py')

In [ ]:
best_tpot=tpot.predict(test_X_NotText)
print(metrics.classification_report([class_mapping[letter] for letter in test_y_NotText],[class_mapping[letter] for letter in best_tpot]))

               precision    recall  f1-score   support

Acoustic/Folk       0.64      0.55      0.59       532
          Alt       0.52      0.41      0.46       394
    Alt_Music       0.34      0.28      0.31       767
        Blues       0.93      0.94      0.93       172
      Country       0.46      0.65      0.54      1463
       HipHop       0.73      0.74      0.73       465
        Indie       0.16      0.03      0.05       402
 Instrumental       0.53      0.52      0.53       771
        Metal       0.67      0.71      0.69       116
          Pop       0.76      0.80      0.78       121
         Rock       0.70      0.74      0.72       196

     accuracy                           0.53      5399
    macro avg       0.59      0.58      0.58      5399
 weighted avg       0.51      0.53      0.51      5399



In [ ]:
tpot.pareto_front_fitted_pipelines_

{'XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=9, XGBClassifier__min_child_weight=6, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.4, XGBClassifier__verbosity=0)': Pipeline(memory=None,
          steps=[('xgbclassifier',
                  XGBClassifier(base_score=0.5, booster='gbtree',
                                colsample_bylevel=1, colsample_bynode=1,
                                colsample_bytree=1, gamma=0, gpu_id=-1,
                                importance_type='gain',
                                interaction_constraints='', learning_rate=0.1,
                                max_delta_step=0, max_depth=9,
                                min_child_weight=6, missing=nan,
                                monotone_constraints='()', n_estimators=100,
                                n_jobs=1, num_parallel_tree=1,
                                objective='multi:softprob', random_state=0,
        

## Second Try

In [ ]:
tpot2 = TPOTClassifier(
    #config_dict='TPOT sparse',
    generations=5, population_size=10,
    scoring = 'f1_macro',
    max_time_mins = 30, 
    n_jobs = -1,
    early_stop=10,
    verbosity = 3,
    cv = 3,
    random_state=0
    )

In [ ]:
tpot2.fit(train_X_NotText, train_y_NotText)

32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/10 [00:00<?, ?pipeline/s]

_pre_test decorator: _random_mutation_operator: num_test=0 '(slice(None, None, None), 0)' is an invalid key.

Generation 1 - Current Pareto front scores:

-1	0.5369823001841615	RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=False, RandomForestClassifier__criterion=entropy, RandomForestClassifier__max_features=0.4, RandomForestClassifier__min_samples_leaf=18, RandomForestClassifier__min_samples_split=16, RandomForestClassifier__n_estimators=100)

Generation 2 - Current Pareto front scores:

-1	0.5385261601791083	RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.8500000000000001, RandomForestClassifier__min_samples_leaf=9, RandomForestClassifier__min_samples_split=17, RandomForestClassifier__n_estimators=100)
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.

Generation 3 - Cu

TPOTClassifier(config_dict=None, crossover_rate=0.1, cv=3,
               disable_update_check=False, early_stop=10, generations=5,
               log_file=None, max_eval_time_mins=5, max_time_mins=30,
               memory=None, mutation_rate=0.9, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=10,
               random_state=0, scoring='f1_macro', subsample=1.0, template=None,
               use_dask=False, verbosity=3, warm_start=False)

In [ ]:
pd.DataFrame(tpot2.evaluated_individuals_).T.reset_index().sort_values(by='internal_cv_score',ascending=False)

,index,generation,mutation_count,crossover_count,predecessor,operator_count,internal_cv_score
47,"RandomForestClassifier(RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=Tr...",4,2,0,"(RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=False, RandomForestClass...",2,0.547195
32,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=4, XGBCla...",3,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=4, XGBCl...",1,0.545344
45,"ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__...",4,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.543219
55,"ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier__...",5,3,0,"(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier_...",1,0.54296
51,"RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassif...",5,3,0,"(RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassi...",1,0.542538
43,"XGBClassifier(GradientBoostingClassifier(input_matrix, GradientBoostingClassifier__learning_rate...",4,3,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=4, XGBCl...",2,0.542461
57,"RandomForestClassifier(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False,...",5,3,0,"(RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassi...",2,0.542258
56,"ExtraTreesClassifier(MinMaxScaler(input_matrix), ExtraTreesClassifier__bootstrap=False, ExtraTre...",5,3,0,"(ExtraTreesClassifier(input_matrix, ExtraTreesClassifier__bootstrap=False, ExtraTreesClassifier_...",2,0.541413
37,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=5, XGBCla...",3,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.539518
50,"RandomForestClassifier(ZeroCount(MLPClassifier(input_matrix, MLPClassifier__alpha=0.01, MLPClass...",5,4,0,"(RandomForestClassifier(ZeroCount(input_matrix), RandomForestClassifier__bootstrap=True, RandomF...",3,0.538872


In [ ]:
tpot2.export('/content/drive/MyDrive/Sistemas Inteligentes/BestModel_gen5_pop10.py')

In [ ]:
best_tpot2=tpot2.predict(test_X_NotText)
print(metrics.classification_report([class_mapping[letter] for letter in test_y_NotText],[class_mapping[letter] for letter in best_tpot2]))

               precision    recall  f1-score   support

Acoustic/Folk       0.59      0.60      0.59       532
          Alt       0.47      0.42      0.44       394
    Alt_Music       0.31      0.27      0.29       767
        Blues       0.92      0.92      0.92       172
      Country       0.47      0.52      0.50      1463
       HipHop       0.69      0.73      0.71       465
        Indie       0.05      0.03      0.04       402
 Instrumental       0.51      0.56      0.53       771
        Metal       0.71      0.66      0.68       116
          Pop       0.80      0.78      0.79       121
         Rock       0.68      0.76      0.71       196

     accuracy                           0.50      5399
    macro avg       0.56      0.57      0.56      5399
 weighted avg       0.49      0.50      0.49      5399



In [ ]:
tpot2.pareto_front_fitted_pipelines_

{'RandomForestClassifier(RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassifier__criterion=gini, RandomForestClassifier__max_features=0.25, RandomForestClassifier__min_samples_leaf=2, RandomForestClassifier__min_samples_split=11, RandomForestClassifier__n_estimators=100), RandomForestClassifier__bootstrap=False, RandomForestClassifier__criterion=entropy, RandomForestClassifier__max_features=0.4, RandomForestClassifier__min_samples_leaf=18, RandomForestClassifier__min_samples_split=16, RandomForestClassifier__n_estimators=100)': Pipeline(memory=None,
          steps=[('stackingestimator',
                  StackingEstimator(estimator=RandomForestClassifier(bootstrap=True,
                                                                     ccp_alpha=0.0,
                                                                     class_weight=None,
                                                                     criterion='gini',
                

## Third Try

In [ ]:
tpot3 = TPOTClassifier(
    #config_dict='TPOT sparse',
    generations=5, population_size=5,
    crossover_rate=0.5, mutation_rate=0.5,
    scoring = 'f1_macro',
    max_time_mins = 30, 
    n_jobs = -1,
    early_stop=10,
    verbosity = 3,
    cv = 3,
    random_state=0
    )

In [ ]:
tpot3.fit(train_X_NotText, train_y_NotText)

32 operators have been imported by TPOT.


Optimization Progress:   0%|          | 0/5 [00:00<?, ?pipeline/s]


Generation 1 - Current Pareto front scores:

-1	0.5349034055215142	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBClassifier__min_child_weight=7, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.4, XGBClassifier__verbosity=0)
Invalid pipeline encountered. Skipping its evaluation.

Generation 2 - Current Pareto front scores:

-1	0.5368539212461337	XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBClassifier__min_child_weight=7, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.45, XGBClassifier__verbosity=0)
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.

Generation 3 - Current Pareto front scores:

-1	0.54777

TPOTClassifier(config_dict=None, crossover_rate=0.5, cv=3,
               disable_update_check=False, early_stop=10, generations=5,
               log_file=None, max_eval_time_mins=5, max_time_mins=30,
               memory=None, mutation_rate=0.5, n_jobs=-1, offspring_size=None,
               periodic_checkpoint_folder=None, population_size=5,
               random_state=0, scoring='f1_macro', subsample=1.0, template=None,
               use_dask=False, verbosity=3, warm_start=False)

In [ ]:
pd.DataFrame(tpot3.evaluated_individuals_).T.reset_index().sort_values(by='internal_cv_score',ascending=False)

,index,generation,mutation_count,crossover_count,predecessor,operator_count,internal_cv_score
13,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCla...",3,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.547779
22,"XGBClassifier(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, PolynomialFeatures_...",5,3,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBCl...",2,0.539481
20,"RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=True, RandomForestClassif...",5,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.538084
11,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCla...",2,1,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.536854
16,"XGBClassifier(MinMaxScaler(input_matrix), XGBClassifier__learning_rate=0.1, XGBClassifier__max_d...",4,2,1,"(XGBClassifier(MinMaxScaler(input_matrix), XGBClassifier__learning_rate=0.1, XGBClassifier__max_...",2,0.536463
14,"XGBClassifier(MinMaxScaler(input_matrix), XGBClassifier__learning_rate=0.1, XGBClassifier__max_d...",3,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",2,0.535956
18,"XGBClassifier(RandomForestClassifier(input_matrix, RandomForestClassifier__bootstrap=False, Rand...",4,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",2,0.535717
2,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCla...",0,0,0,"(ROOT,)",1,0.534903
21,"XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCla...",5,2,0,"(XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.1, XGBClassifier__max_depth=2, XGBCl...",1,0.534212
6,"GradientBoostingClassifier(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, Polyno...",1,1,0,"(GaussianNB(PolynomialFeatures(input_matrix, PolynomialFeatures__degree=2, PolynomialFeatures__i...",2,0.533859


In [ ]:
tpot3.export('/content/drive/MyDrive/Sistemas Inteligentes/BestModel_gen5_pop5_crossequalmut.py')

In [ ]:
best_tpot3=tpot3.predict(test_X_NotText)
print(metrics.classification_report([class_mapping[letter] for letter in test_y_NotText],[class_mapping[letter] for letter in best_tpot3]))

               precision    recall  f1-score   support

Acoustic/Folk       0.61      0.56      0.58       532
          Alt       0.51      0.42      0.46       394
    Alt_Music       0.38      0.28      0.32       767
        Blues       0.90      0.91      0.91       172
      Country       0.47      0.67      0.56      1463
       HipHop       0.71      0.69      0.70       465
        Indie       0.26      0.03      0.05       402
 Instrumental       0.51      0.55      0.53       771
        Metal       0.71      0.71      0.71       116
          Pop       0.71      0.76      0.74       121
         Rock       0.69      0.72      0.70       196

     accuracy                           0.54      5399
    macro avg       0.59      0.57      0.57      5399
 weighted avg       0.52      0.54      0.51      5399



In [ ]:
tpot3.pareto_front_fitted_pipelines_

{'XGBClassifier(input_matrix, XGBClassifier__learning_rate=0.5, XGBClassifier__max_depth=2, XGBClassifier__min_child_weight=7, XGBClassifier__n_estimators=100, XGBClassifier__n_jobs=1, XGBClassifier__subsample=0.45, XGBClassifier__verbosity=0)': Pipeline(memory=None,
          steps=[('xgbclassifier',
                  XGBClassifier(base_score=0.5, booster='gbtree',
                                colsample_bylevel=1, colsample_bynode=1,
                                colsample_bytree=1, gamma=0, gpu_id=-1,
                                importance_type='gain',
                                interaction_constraints='', learning_rate=0.5,
                                max_delta_step=0, max_depth=2,
                                min_child_weight=7, missing=nan,
                                monotone_constraints='()', n_estimators=100,
                                n_jobs=1, num_parallel_tree=1,
                                objective='multi:softprob', random_state=0,
       